In [1]:
import os
import psycopg2

# Create postgres database
conn = psycopg2.connect(os.environ.get('DATABASE_URL'))
cursor = conn.cursor()


In [6]:
states = [
    'limbo',
    'memo_submitted',
    'memo_edits_required',
    'memo_dropped',
    'synmemo_baking',
    'synmemo_edits_required',
    'synmemo_dropped',
    'synmemo_submitted',
    'synthesis_submitted',
    'distribution_baking',
    'distribution_shipping'
]

create_db = """
BEGIN;

-- Get those UUIDs
CREATE EXTENSION IF NOT EXISTS "uuid-ossp";

-- Create Update Function

CREATE TABLE memos (
    id uuid PRIMARY KEY DEFAULT uuid_generate_v4(),
    name TEXT NOT NULL,
    short_description TEXT NOT NULL,
    justification TEXT NOT NULL,
    priority SMALLINT NOT NULL,
    urgency SMALLINT NOT NULL,
    estimated_gene_count SMALLINT NOT NULL,
    memo_link TEXT,
    synmemo_link TEXT,
    memo BYTEA,
    synmemo BYTEA,
    doi TEXT
);

CREATE TABLE authors (
    name TEXT PRIMARY KEY,
    contact_details TEXT
);

CREATE TABLE institutions (
    name TEXT PRIMARY KEY,
    master_signed BOOLEAN NOT NULL
);

CREATE TABLE labs (
    name TEXT PRIMARY KEY,
    institution TEXT NOT NULL REFERENCES institutions(name)
);

CREATE TABLE memos_authors_labs (
    memo uuid NOT NULL REFERENCES memos(id),
    author TEXT NOT NULL REFERENCES authors(name),
    lab TEXT NOT NULL REFERENCES labs(name),
    position SMALLINT NOT NULL,
    PRIMARY KEY (memo,author,lab)
);

CREATE TABLE literature_search (
    id uuid PRIMARY KEY DEFAULT uuid_generate_v4(),
    memo uuid NOT NULL REFERENCES memos(id),
    doi TEXT NOT NULL,
    notes TEXT NOT NULL
);

CREATE TABLE timestamps (
    memo uuid NOT NULL REFERENCES memos(id),
    state TEXT NOT NULL CHECK (state in ('{}')),
    created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    notes TEXT,
    PRIMARY KEY (memo, state)
);

COMMIT;
""".format("','".join(states))
#cursor.execute(create_db)
#conn.commit()

DuplicateTable: relation "memos" already exists


In [21]:
cursor.execute('ROLLBACK')

In [17]:
### Add column
cmd = """
BEGIN;
ALTER TABLE authors
DROP COLUMN lab ;
COMMIT;
"""
cursor.execute(cmd)
conn.commit()

In [2]:
cmd = """
BEGIN;

COMMIT;
"""
cursor.execute(cmd)
conn.commit()

In [6]:
### Manipulate memo_state_check
states = [
    'limbo',
    'memo_submitted',
    'memo_edits_required',
    'memo_dropped',
    'synmemo_baking',
    'synmemo_edits_required',
    'synmemo_dropped',
    'synmemo_submitted',
    'synthesis_submitted',
    'distribution_baking',
    'distribution_shipping'
]

cmd = """
BEGIN;
ALTER TABLE timestamps DROP CONSTRAINT timestamps_state_check;
ALTER TABLE timestamps ADD CONSTRAINT timestamps_state_check CHECK (state in ('{}'));
COMMIT;
""".format("','".join(states))
#cursor.execute(cmd)
#conn.commit()

In [7]:
print(cmd)


BEGIN;
ALTER TABLE timestamps DROP CONSTRAINT timestamps_state_check;
ALTER TABLE timestamps ADD CONSTRAINT timestamps_state_check CHECK (state in ('limbo','memo_submitted','memo_edits_required','memo_dropped','synmemo_baking','synmemo_edits_required','synmemo_dropped','synmemo_submitted','synthesis_submitted','distribution_baking','distribution_shipping'));
COMMIT;



In [18]:
### Add table
cmd = """
BEGIN;
CREATE TABLE timestamps (
    memo uuid NOT NULL REFERENCES memos(id),
    state TEXT NOT NULL CHECK (state in ('{}')),
    created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    PRIMARY KEY (memo, state)
);
COMMIT;
""".format("','".join(states))
#cursor.execute(cmd)
#conn.commit()

In [5]:
cursor.execute("ROLLBACK")